# LDA

In [1]:
#logistic regression

In [12]:
seed= 123

In [19]:
import pandas as pd
import numpy as np 
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier as Ada
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import train_test_split

In [15]:
def get_train_test_data(year = 2018):
    train_data = pd.read_csv('../ouput/train_'+str(year)+'.csv')

    train_data['WTseed']= train_data['WTseed'].fillna(18)####
    train_data['LTseed']= train_data['LTseed'].fillna(18)####

    train_data['ID'] = train_data.apply(lambda r: '_'.join(map(str, [r['Season']]+sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
    train_data['IDTeams'] = train_data.apply(lambda r: '_'.join(map(str, sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
    train_data['Team1'] = train_data.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[0], axis=1)
    train_data['Team2'] = train_data.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[1], axis=1)
    train_data['IDTeam1'] = train_data.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
    train_data['IDTeam2'] = train_data.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
    train_data['Pred'] = train_data.apply(lambda r: 1. if sorted([r['WTeamID'],r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1)
    train_data['ScoreDiff'] = train_data['WScore']-train_data['LScore']
    train_data['ScoreDiff'] = train_data.apply(lambda r: r['ScoreDiff'] * -1 if r['Pred'] == 0. else r['ScoreDiff'], axis=1)
    train_data['Team1Score'] = train_data.apply(lambda r: r['WScore'] if r['Pred'] == 1. else r['LScore'],axis=1)
    train_data['Team2Score'] = train_data.apply(lambda r: r['WScore'] if r['Pred'] == 0. else r['LScore'],axis=1)
    train_data['Team1Seed'] = train_data.apply(lambda r: r['WTseed']if r['Pred'] == 1. else r['LTseed'],axis=1)
    train_data['Team2Seed'] = train_data.apply(lambda r: r['WTseed']if r['Pred'] == 0. else r['LTseed'],axis=1)
    train_data['SeedDiff'] = train_data.apply(lambda r: r['Team1Seed']-r['Team2Seed'],axis = 1)
    train_data['Team1ScoreRatio'] = train_data['Team1Score']/(train_data['Team1Score']+train_data['Team2Score'])
    train_data['Team2ScoreRatio'] = train_data['Team2Score']/(train_data['Team1Score']+train_data['Team2Score'])
    train_data['Team1Loc'] = train_data.apply(lambda r: r['WLoc'] if r['Pred'] == 1 else None,axis=1)
    train_data['Team2Loc'] = train_data.apply(lambda r: r['WLoc'] if r['Pred'] == 0 else None,axis=1)
    train_data['Team1Loc'] = train_data[['Team1Loc','Team2Loc']].apply(lambda r: 'N' if r[1]=='N' else r[0],axis=1)
    train_data['Team2Loc'] = train_data[['Team1Loc','Team2Loc']].apply(lambda r: 'N' if r[0]=='N' else r[1],axis=1)
    train_data['Team1Loc'] = train_data[['Team1Loc','Team2Loc']].apply(lambda r: 'A' if r[1]=='H' else r[0],axis=1)
    train_data['Team1Loc'] = train_data[['Team1Loc','Team2Loc']].apply(lambda r: 'H' if r[1]=='A' else r[0],axis=1)
    train_data['Team2Loc'] = train_data[['Team1Loc','Team2Loc']].apply(lambda r: 'A' if r[0]=='H' else r[1],axis=1)
    train_data['Team2Loc'] = train_data[['Team1Loc','Team2Loc']].apply(lambda r: 'H' if r[0]=='A' else r[1],axis=1)
    
    train_data = train_data.drop(['index','DayNum','WScore','LScore','WTseed','LTseed','WTeamID','LTeamID'],axis=1)


    team1d_score_spread = train_data.groupby(['Season', 'Team1'])[['ScoreDiff', 'Team1Score','Team1ScoreRatio']].mean().reset_index()\
    .set_index('Season').rename(columns = {'ScoreDiff' : 'ScoreDiff1','Team1ScoreRatio':'Team1ScoreRatio_avg','Team1Score':'Team1Score_avg'})
    team2d_score_spread = train_data.groupby(['Season', 'Team2'])[['ScoreDiff', 'Team2Score','Team2ScoreRatio']].mean().reset_index()\
    .set_index('Season').rename(columns = {'ScoreDiff' : 'ScoreDiff2','Team2ScoreRatio':'Team2ScoreRatio_avg','Team2Score':'Team2Score_avg'})
    score_spread = team1d_score_spread.join(team2d_score_spread).reset_index()

    X_train = pd.merge(train_data,score_spread,on=['Team1','Team2','Season'])

    y_train = X_train['Pred']
    X_train = X_train.drop(['WLoc','NumOT','Pred','Team1Score','Team2Score','Team1ScoreRatio','Team2ScoreRatio','ScoreDiff'],axis=1)
    return X_train,y_train

In [17]:
def train_process1(df):
    df =df.drop(['Unnamed: 0',"Season","ID","IDTeams","Team1","Team2","IDTeam1","IDTeam2"],axis=1)
    Lsea=pd.get_dummies(df["Lsea"],prefix="Lsea")
    Wsea=pd.get_dummies(df["Wsea"],prefix="Wsea")
    df=pd.concat([df,Lsea,Wsea],axis=1)
    df["Team1Loc"]=df["Team1Loc"].astype("category")
    df["Team2Loc"]=df["Team2Loc"].astype("category")
    df.Team1Loc=df.Team1Loc.cat.rename_categories([0,2,1])
    df.Team2Loc=df.Team2Loc.cat.rename_categories([0,2,1])
    df=df.drop(["Lsea","Wsea"],axis=1)
    return df

In [16]:
X_train,y_train=get_train_test_data()

In [18]:
X_train=train_process1(X_train)

In [21]:
#train,val
x_tr, x_val ,y_tr,y_val=train_test_split(X_train,y_train,test_size=0.1,random_state=seed)

In [40]:
def gridcv(estimator,X,y,params):
    
    gridsearch = GridSearchCV(estimator=estimator,
                              param_grid=params,
                              scoring="roc_auc",
                              n_jobs=2,
                              iid=False,
                              cv=5)
    
    gridsearch.fit(X,y)
    
    return gridsearch.grid_scores_, gridsearch.best_params_, gridsearch.best_score_

In [48]:
#model

#logistic regression
logreg = LogisticRegression(penalty='l2',
                            random_state=seed, 
                            solver='sag',
                            max_iter=160,
                            )
#SVM
svm_cf =SVC(C=1.0, 
            kernel='rbf', 
            degree=3, 
            gamma='auto', 
            coef0=0.0, 
            shrinking=True, 
            probability=False, 
            tol=0.001, 
            cache_size=200, 
            class_weight=None, verbose=False, 
            max_iter=-1, 
            decision_function_shape='ovr', 
            random_state=seed)

#Adaboost
ada_cf = Ada(base_estimator=None, 
             n_estimators=50, 
             learning_rate=1.0, 
             algorithm='SAMME.R', 
             random_state=seed)

In [ ]:
#parameters tuning
params ={
}
gridcv(svm_cf,x_tr,y_tr,params)